<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Загрузка-и-прочтение-данных" data-toc-modified-id="Загрузка-и-прочтение-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка и прочтение данных</a></span></li><li><span><a href="#Предобработка" data-toc-modified-id="Предобработка-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Предобработка</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Сознание-признаков-через-TF-IDF" data-toc-modified-id="Сознание-признаков-через-TF-IDF-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Сознание признаков через TF-IDF</a></span></li><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li><li><span><a href="#LGBMClassifier" data-toc-modified-id="LGBMClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>LGBMClassifier</a></span></li></ul></li><li><span><a href="#Общий-вывод:" data-toc-modified-id="Общий-вывод:-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Общий вывод:</a></span></li></ul></div>

# Классификация токсичных комментариев

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 

*План работы:*

1. Загрузим и подготовим данные.
2. Обучим разные модели. 
3. Сделаем выводы.

## Подготовка

Для начала импортируем все нужные библиотеки и функции для работы.

In [1]:
!pip install -U scikit-learn
!pip3 install catboost
!pip install lightgbm

import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords as nltk_stopwords 
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize,pos_tag

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import f1_score

from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
stopwords = set(nltk_stopwords.words('english'))

Requirement already up-to-date: scikit-learn in c:\users\user\anaconda3\lib\site-packages (1.0.1)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Загрузка и прочтение данных

In [2]:
df = pd.read_csv('toxic_comments.csv')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Входные данные представялют собой таблицу, в которой каждая строка это наблюдение за одним комментарием. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак; столбец бинарный, а значит в нем *1* означает "да", а *0* означает "нет". 

Сразу же видно, что в данных нет пропусков.

### Предобработка

Проверим наличие полных дубликатов.

In [3]:
df.duplicated().sum()

0

Теперь стоит проверить целевой признак на наличие аномальных значений.

In [4]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Отлично, столбец бинарный.

### Лемматизация

Теперь лемматизируем тексты, чтобы потом можно было подготовить признаки.

In [5]:
def lemmatize(text):
    '''
    Функция принимает строку, оставялет в ней только англицские буквы и потом находит из оставшего текста леммы
    '''
    #с помощью регуляки оставим только английские буквы
    re_text = re.sub(r'[^a-zA-Z]', ' ', text.lower())
    clear_re =  ' '.join(re_text.split())
    
    #возьмем английский лемматизатор
    wordnet = WordNetLemmatizer()
    
    lemm_list = []
    for token,tag in pos_tag(word_tokenize(clear_re)):
        pos=tag[0].lower()
        
        if pos not in ['a', 'r', 'n', 'v']:
            pos='n'
        
        lemm_list.append(wordnet.lemmatize(token,pos))
    
    lemm_text = " ".join(lemm_list)
    
    return lemm_text

In [6]:
%%time
clear_text = df['text'].apply(lemmatize)

Wall time: 12min 51s


In [7]:
print(df['text'].head())
print('------------------------------------------------------')
print(clear_text.head())

0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
3    "\nMore\nI can't make any real suggestions on ...
4    You, sir, are my hero. Any chance you remember...
Name: text, dtype: object
------------------------------------------------------
0    explanation why the edits make under my userna...
1    d aww he match this background colour i m seem...
2    hey man i m really not try to edit war it s ju...
3    more i can t make any real suggestion on impro...
4    you sir be my hero any chance you remember wha...
Name: text, dtype: object


### Сознание признаков через TF-IDF

Сначала разделим выборки.

In [8]:
train_x, test_x = train_test_split(clear_text, test_size=0.25, random_state=5)

train_y = df.loc[train_x.index, 'toxic']
test_y = df.loc[test_x.index, 'toxic']

Теперь через счётчик величин TF-IDF преобразуем лемматизированный текст в векторную форму, понятную алгоритам ML.

In [9]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

count_tf_idf.fit(train_x)
train_tfidf = count_tf_idf.transform(train_x)
test_tfidf = count_tf_idf.transform(test_x)

### Вывод:

Данные были успешно загружены. Данные были хорошо собраны: нет пропущенных значений, нет дубликатов, нет аномалий.

В ходе подготовки признаков для дальнейшего обучения, текстовые столбец был лемматизирован, а затем преобразован в векторную форму с помощью TF-IDF. На выходе мы получили готовые обучающую и тестовую выборки.

## Обучение

Для обучения попробуем 3 модели: 
 1. LogisticRegression
 2. CatBoostClassifier
 3. LGBMClassifier
 
Для каждой модели подберем лучшие параметры на кроссвалидации. Затем модели с лучшими параметрами протестируем на тестовой выборке и вычислиним для них результаты метрики качества F1.

### LogisticRegression

In [10]:
%%time 

model_lr = LogisticRegression(random_state=5)
params_lr = {'class_weight': ['balanced', None], 'C':np.arange(1, 10)}
grid_lr = GridSearchCV(model_lr, params_lr, cv=3, n_jobs=-1)

grid_lr.fit(train_tfidf, train_y)

print('Лучшие параметры для LogisticRegression', grid_lr.best_params_, end='\n\n')

Лучшие параметры для LogisticRegression {'C': 7, 'class_weight': None}

Wall time: 3min 46s


C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Протестируем параметры.

In [11]:
model_lr_test = LogisticRegression(class_weight=None , C=7, random_state=5)
model_lr_test.fit(train_tfidf, train_y)
preds_lr = model_lr_test.predict(test_tfidf)
lr_f1_test = f1_score(test_y, preds_lr)

C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
lr_f1_test

0.7695307060298009

### CatBoostClassifier

Из-за нехватки мощности код ниже выполнялся в Google Colab.


```python
model_cat = CatBoostClassifier(loss_function='Logloss', iterations=150, random_seed=5, verbose=False)
parameters_cat = {'depth':[3, 5, 10], 'learning_rate':np.arange(0.1, 1, 0.2)}
cat_grid = GridSearchCV(model_cat, parameters_cat, cv=3, n_jobs=-1)

cat_grid.fit(train_tfidf, train_y)

print('Лучшие параметры для CatBoost', cat_grid.best_params_, end='\n\n')
```

Лучшие параметры для CatBoost {'depth': 5, 'learning_rate': 0.5}

Протестируем их.

In [13]:
%%time

model_cat_test = CatBoostClassifier(loss_function='Logloss', iterations=150, random_seed=5, verbose=False,
                                  learning_rate=0.5, depth=5)
model_cat_test.fit(train_tfidf, train_y)

cat_preds = model_cat_test.predict(test_tfidf)

cat_f1_test = f1_score(test_y, cat_preds) 

Wall time: 1min 58s


In [14]:
cat_f1_test

0.7472622478386166

### LGBMClassifier

Из-за нехватки мощности код ниже выполнялся в Google Colab

```python
model_lgbm = LGBMClassifier(boosting_type='gbdt') 

params_lgbm = {'num_leaves':[31, 50, 100], 'learning_rate':[0.1, 0.4, 0.7]}
grid_lgbm = GridSearchCV(model_lgbm, params_lgbm, cv=3, n_jobs=-1)

grid_lgbm.fit(train_tfidf, train_y)

print('Лучшие параметры для модели LightGBM', grid_lgbm.best_params_, end='\n\n')
```

Лучшие параметры для модели LightGBM: {'num_leaves': 5, 'learning_rate': 0.1}

Тестируем.

In [15]:
%%time

model_lgbm_test = LGBMClassifier(boosting_type='gbdt', learning_rate=0.1, num_leaves=100)
model_lgbm_test.fit(train_tfidf, train_y)

lgbm_preds = model_lgbm_test.predict(test_tfidf)

lgbm_f1_test = f1_score(test_y, lgbm_preds) 

Wall time: 58.6 s


In [16]:
lgbm_f1_test

0.774452756996398

## Общий вывод:

Входные данные представляли собой таблицу из двух колонок: в первой был текст комментария, а во второй - бинарно представлял токсичен комментарий или нет.

Данные были отлично собранны. В них отсутствовали пропуски и дубликаты. Аномалий тоже не обнаружено.

Тексты комментариев были лемматизированы, а затем переведены в векторные представляения с помощью техники TF-IDF.

Были опробованы 3 модели ML: *LogisticRegression, CatBoostClassifier и LGBMClassifier*. Для каждой модели были подобраны лучшие парметры на кросс-валидации.

По итогам проверки на тестовой выборке только модели LogisticRegression и LGBMClassifier смогли преодолеть минимальный порог метрики качества. 

Поэтому лучшей моделью для определения токсичности комментариев признается **модель LGBMClassifier с гиперпараметрами:  boosting_type='gbdt', learning_rate=0.1, num_leaves=100; и результатом F1 = 0.774452756996398**